In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import mailbox

In [3]:

mailboxfile = "C:/Users/HP/takeout-20220527T154953Z-001/Takeout/Mail/All mail Including Spam and Trash.mbox"

mbox = mailbox.mbox(mailboxfile)
mbox

In [4]:
for key in mbox[0].keys():
    print(key)

X-GM-THRID
X-Gmail-Labels
Delivered-To
Received
X-Received
ARC-Seal
ARC-Message-Signature
ARC-Authentication-Results
Return-Path
Received
Received-SPF
Authentication-Results
DKIM-Signature
X-Google-DKIM-Signature
X-Gm-Message-State
X-Google-Smtp-Source
MIME-Version
X-Received
Date
List-Unsubscribe
X-Notifications
X-Notifications-Bounce-Info
Message-ID
Subject
From
To
Content-Type


In [5]:
import csv

In [6]:
with open('mailbox.csv', 'w') as outputfile:
    writer = csv.writer(outputfile)
    writer.writerow(['subject','from','date','to','label','thread'])
    for message in mbox:
        writer.writerow([
            message['subject'],
            message['from'],
            message['date'],
            message['to'],
            message['X-Gmail-Labels'],
            message['X-GM-THRID']
        ])
        

In [7]:
dfs = pd.read_csv('mailbox.csv', names=['subject', 'from', 'date', 'to',
'label', 'thread'])

In [8]:
dfs.head(5)

,subject,from,date,to,label,thread
0,subject,from,date,to,label,thread
1,=?UTF-8?Q?3_new_jobs_for_=E2=80=98data_scienti...,Job alerts from Google <notify-noreply@google....,"Tue, 24 May 2022 12:27:26 -0700",ayosamobitade@gmail.com,"Inbox,Opened,Category Updates",1733736999130792636
2,Your program recommendations for the week,"""Will at edX"" <edX@news.edx.org>","Thu, 26 May 2022 14:00:10 +0000",ayosamobitade@gmail.com,"Inbox,Category Promotions,Unread",1733897612589550289
3,=?utf-8?Q?Excel=20Newsletter=20=2D=20STOP=20Em...,=?utf-8?Q?My=20Online=20Training=20Hub?= <webs...,"Thu, 26 May 2022 12:06:07 +0000",<Ayosamobitade@gmail.com>,"Inbox,Category Updates,Unread",1733890522517290196
4,=?UTF-8?Q?*Konga_Food_is_now_live_in_Lagos!*_=...,Konga Food <Newsletter@info.konga.com>,"Mon, 23 May 2022 15:25:50 +0530",ayosamobitade@gmail.com,"Inbox,Category Promotions,Unread",1733610439749409235


In [9]:
dfs.dtypes

subject    object
from       object
date       object
to         object
label      object
thread     object
dtype: object

In [10]:
dfs['date'] = dfs['date'].apply(lambda x: pd.to_datetime(x, errors='coerce', utc=True))

C:\Users\HP\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\Users\HP\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\Users\HP\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [11]:
dfs.dtypes

subject                 object
from                    object
date       datetime64[ns, UTC]
to                      object
label                   object
thread                  object
dtype: object

In [12]:
## Removing the Nan values

In [13]:
dfs = dfs[dfs['date'].notna()]

In [14]:
## Applying descriptive statistics

In [15]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21565 entries, 1 to 21566
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype              
---  ------   --------------  -----              
 0   subject  21502 non-null  object             
 1   from     21565 non-null  object             
 2   date     21565 non-null  datetime64[ns, UTC]
 3   to       21545 non-null  object             
 4   label    21565 non-null  object             
 5   thread   21565 non-null  object             
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 1.2+ MB


In [16]:
# Note that ther are 21565 emails

In [17]:
dfs.head(10)

,subject,from,date,to,label,thread
1,=?UTF-8?Q?3_new_jobs_for_=E2=80=98data_scienti...,Job alerts from Google <notify-noreply@google....,2022-05-24 19:27:26+00:00,ayosamobitade@gmail.com,"Inbox,Opened,Category Updates",1733736999130792636
2,Your program recommendations for the week,"""Will at edX"" <edX@news.edx.org>",2022-05-26 14:00:10+00:00,ayosamobitade@gmail.com,"Inbox,Category Promotions,Unread",1733897612589550289
3,=?utf-8?Q?Excel=20Newsletter=20=2D=20STOP=20Em...,=?utf-8?Q?My=20Online=20Training=20Hub?= <webs...,2022-05-26 12:06:07+00:00,<Ayosamobitade@gmail.com>,"Inbox,Category Updates,Unread",1733890522517290196
4,=?UTF-8?Q?*Konga_Food_is_now_live_in_Lagos!*_=...,Konga Food <Newsletter@info.konga.com>,2022-05-23 09:55:50+00:00,ayosamobitade@gmail.com,"Inbox,Category Promotions,Unread",1733610439749409235
5,30+ new jobs for 'data scientist',LinkedIn Job Alerts <jobalerts-noreply@linkedi...,2022-05-23 15:04:34+00:00,"""Ayobami Samuel Obitade B.Tech"" <ayosamobitade...","Inbox,Category Updates,Unread",1733629866042045910
6,Re: [DataTalksClub/data-engineering-zoomcamp] ...,Alexey Grigorev <notifications@github.com>,2022-05-22 20:53:24+00:00,DataTalksClub/data-engineering-zoomcamp <data-...,"Trash,Opened,Category Forums",1733323756223473963
7,Re: [DataTalksClub/data-engineering-zoomcamp] ...,Alexey Grigorev <notifications@github.com>,2022-05-22 20:53:22+00:00,DataTalksClub/data-engineering-zoomcamp <data-...,"Trash,Opened,Category Forums",1733323756223473963
8,[DataTalksClub/data-engineering-zoomcamp] Time...,Alexey Grigorev <notifications@github.com>,2022-05-20 05:59:07+00:00,DataTalksClub/data-engineering-zoomcamp <data-...,"Trash,Opened,Category Updates",1733323756223473963
9,=?utf-8?q?I_was_refused_a_USA_visa_on_a_counte...,Quora Digest <english-personalized-digest@quor...,2022-05-22 04:12:22+00:00,ayosamobitade@gmail.com,"Inbox,Category Updates,Unread",1733498234232534466
10,GRE Question of the Day for May 15,"""Kaplan Test Prep"" <kaplantestprep@mail.kaptes...",2022-05-15 13:35:13+00:00,<ayosamobitade@gmail.com>,"Inbox,Category Promotions,Unread",1732899544725135310


In [18]:
## Data refactoring

In [19]:
# creating an email extractor

import re

def extract_email_address(string):
    email = re.findall(r'<(.+?)>', string)
    if not email:
        email = list(filter(lambda y: '@' in y, string.split()))
        return email[0] if email else np.nan


In [20]:
# applying the email extractor

dfs['from'] = dfs['from'].apply(lambda x: extract_email_address(x))

In [21]:
dfs.sample(10)

,subject,from,date,to,label,thread
263,Microsoft Build digital venue is now open!,None,2022-05-17 15:25:19+00:00,ayosamobitade@gmail.com,"Inbox,Category Updates,Unread",1733087720565169540
2783,=?UTF-8?Q?=F0=9F=94=84_Stephy_Pearlz=F0=9F=87=...,None,2022-04-22 18:38:33+00:00,Samworld Press <ayosamobitade@gmail.com>,"Inbox,Category Social,Unread",1730834820896447817
6239,"Jobs report: officer, mining, developer ... in...",None,2021-04-16 07:49:30+00:00,Obitade Ayobami <ayosamobitade@gmail.com>,"Inbox,Category Promotions,Unread",1697182511735112518
15832,"Speak a foreign language fluently, Ayobami!",None,2019-08-04 09:52:31+00:00,Ayobami Samuel Obitade <ayosamobitade@gmail.com>,"Inbox,Category Promotions,Unread",1640929549245705334
2468,1 new digital marketer job in Ibadan,None,2021-11-09 11:23:36+00:00,ayosamobitade@gmail.com,"Inbox,Category Updates,Unread",1715950146028977097
14293,Your paperback book is available in the Amazon...,None,2020-02-21 12:55:01+00:00,ayosamobitade@gmail.com,"Inbox,Important,Opened,Category Updates",1659150248056445196
19659,Can you really write a GOOD book in a month?,None,2018-09-06 00:00:10+00:00,ayosamobitade@gmail.com,"Inbox,Category Promotions,Unread",1610814073783964647
11097,=?UTF-8?B?Q2xhc3NpYyBBbmQgSWNvbmljIFN0cmlwZXMg...,None,2020-08-10 12:13:10+00:00,Ayo <Ayosamobitade@gmail.com>,"Inbox,Category Promotions,Unread",1674640460619474972
10675,[Webinar Today] An Educator's Guide to Teachin...,None,2021-03-02 18:30:15+00:00,<ayosamobitade@gmail.com>,"Inbox,Category Promotions,Unread",1693146277508745922
10455,All the Pandas read_csv() you should know to s...,None,2020-09-27 07:40:00+00:00,ayosamobitade@gmail.com,"Inbox,Category Promotions,Unread",1678971930316469169


In [22]:
myemail = 'ayosamobitade@gmail.com'
dfs['label'] = dfs['from'].apply(lambda x: 'sent' if x==myemail else 'inbox')

## Dropping columns


In [24]:
dfs.drop('to', axis = 1, inplace = True)

In [25]:
# adjusting the time zones

import datetime
import pytz

def refaactor_timezone(x):
    est = pytz.timezone('US/Eastern')
    return X.astimezone(est)

dfs['date'] = dfs['date'].apply(lambda x: refactor_timezone(x))



NameError: name 'refactor_timezone' is not defined